In [1]:
# !pip install alpaca-trade-api

In [2]:
import alpaca_trade_api as tradeapi
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
import config
from datetime import datetime, timedelta
import json
import mplfinance as mpf
import os
import pandas as pd
import polars as pl
import plotly.graph_objects as go
import time

In [ ]:
POSITION_FILE: str = 'position.json' 

SYMBOL = 'SPY'

# Initial position dictionary
initial_position_data = {
    "side": "long",
    "entry_price": 180.50,
    "qty": 10,
    "high": 180.50,
    "low": 180.00,
    "entry_time": datetime.now().isoformat(),
    "last_exit": ""
} 

# # Simulated activities
# activities = [{
#         "symbol": SYMBOL,
#         "side": "buy",
#         "transaction_time": datetime.now() - timedelta(hours=2),
#         "realized_pl": 0.00},
#     {
#         "symbol": SYMBOL,
#         "side": "sell",
#         "transaction_time": datetime.now() - timedelta(hours=1, minutes=30),
#         "realized_pl": 15.75},
#     {
#         "symbol": SYMBOL,
#         "side": "buy",
#         "transaction_time": datetime.now() - timedelta(minutes=45),
#         "realized_pl": 0.00},
#     {
#         "symbol": SYMBOL,
#         "side": "sell",
#         "transaction_time": datetime.now() - timedelta(minutes=30),
#         "realized_pl": -5.25}]

In [4]:
def create_position_file_if_not_exists():
    """Creates the position.json file with initial data if it doesn't exist."""
    if not os.path.exists(POSITION_FILE):
        try:
            with open(POSITION_FILE, 'w') as f:
                json.dump(initial_position_data, f, indent=4)
            print(f"'{POSITION_FILE}' not found. Created and initialized it.")
        except Exception as e:
            print(f"Error creating '{POSITION_FILE}': {e}")
    else:
        print(f"'{POSITION_FILE}' already exists.")

In [5]:
create_position_file_if_not_exists()

'position.json' not found. Created and initialized it.


In [6]:
def load_position() -> dict:
    """Loads the trading position from the position.json file."""
    # Ensure the file exists before attempting to load
    create_position_file_if_not_exists()
    try:
        with open(POSITION_FILE, 'r') as f:
            position_data = json.load(f)
            print(f"Successfully loaded position from {POSITION_FILE}")
            return position_data
    except json.JSONDecodeError:
        print(f"Error decoding JSON from {POSITION_FILE}. File might be corrupted.")
        # file is corrupted, lets delete it and re-create
        os.remove(POSITION_FILE)
        print(f"Deleted corrupted {POSITION_FILE}. Re-creating it.")
        try:
             with open(POSITION_FILE, 'r') as f:
                 position_data = json.load(f)
                 return position_data # Should now load the initial data
        except:
             return initial_position_data # Fallback if even re-creation fails (unlikely)
    except Exception as e:
        print(f"An unexpected error occurred while reading {POSITION_FILE}: {e}. Returning initial position.")
        return initial_position_data


In [7]:
load_position()

'position.json' already exists.
Successfully loaded position from position.json


{'side': 'long',
 'entry_price': 180.5,
 'qty': 10,
 'high': 180.5,
 'low': 180.0,
 'entry_time': '2025-06-13T12:55:02.843897',
 'last_exit': ''}

In [8]:
def close_position():
    """Closes the current trading position."""
    try:
        # api.close_position(SYMBOL)
        print("Position closed.")
        # reset_position()
    except Exception as e:
        print(f"Close error: {e}")

In [9]:
def save_position(data):
    """Saves the current trading position to the position.json file."""
    POSITION_FILE = 'position.json'
    with open(POSITION_FILE, 'w') as f:
        json.dump(data, f)
    print("Position saved")

In [10]:
def reset_position():
    """Resets the position by deleting the position.json file."""
    POSITION_FILE = 'position.json'
    if os.path.exists(POSITION_FILE):
        os.remove(POSITION_FILE)
    print("Positioned reset")

In [11]:
reset_position()

Positioned reset


In [12]:
# writte a function that connects tto Alpaca API and returns the api object
def connect_to_alpaca() -> tradeapi.REST | None:
    """Connects to the Alpaca API and returns the API object."""
    api_key = os.getenv("ALPACA_API_KEY")
    api_secret = os.getenv("ALPACA_SECRET_KEY")
    api_url = os.getenv("ALPACA_BASE_URL", "https://paper-api.alpaca.markets")

    if not api_key or not api_secret:
        print("API key and secret must be set in environment variables.")
        return None
    
    try:
        api = tradeapi.REST(
            key_id=api_key,
            secret_key=api_secret,
            base_url=api_url
        )
        print("Connected to Alpaca API")
        return api
    except Exception as e:
        print(f"Error connecting to Alpaca API: {e}")
        return None


In [13]:
api = connect_to_alpaca()
if api is None:
    print("Failed to connect to Alpaca API. Exiting.")
    exit(1)
# get account information
account = api.get_account()
print(f"Account status: {account.status}")
print(f"Account equity: {account.equity}")
print(f"Account buying power: {account.buying_power}")

Connected to Alpaca API
Account status: ACTIVE
Account equity: 99957.69
Account buying power: 360511.34


In [14]:
def fetch_positions(api) -> list:
    """Fetches and returns the current open positions."""
    try:
        positions = api.list_positions()
        if positions:
            for position in positions:
                print(f"Symbol: {position.symbol}, Qty: {position.qty}, Side: {position.side}, Market Value: {position.market_value}")
        else:
            print("No open positions.")
        return positions
    except Exception as e:
        print(f"Error fetching positions: {e}")
        return []

In [15]:
positions = fetch_positions(api)

Symbol: SPY, Qty: 55.459157247, Side: long, Market Value: 33271.057616


In [16]:
def fetch_activities(api, symbol='SPY', after=None) -> list:
    """Fetches and returns account activities for a specific symbol."""
    try:
        activities = api.get_activities(activity_types=['FILL'], after=after)
        symbol_activities = [activity for activity in activities if activity.symbol == symbol]
        if symbol_activities:
            for activity in symbol_activities:
                realized_pl = getattr(activity, "realized_pl", None)
                print(f"Symbol: {activity.symbol}, Side: {activity.side}, Qty: {activity.qty}, Price: {activity.price}, Time: {activity.transaction_time}, P/L: {realized_pl}")
        else:
            print(f"No activities found for symbol {symbol}.")
        return symbol_activities
    except Exception as e:
        print(f"Error fetching activities: {e}")
        return []

In [17]:
# Fetch activities
activities = fetch_activities(api, after=(datetime.now() - timedelta(days=5)).isoformat() + 'Z')
if activities:
    for activity in activities:
        print(f"Recent activity: {activity.side} {activity.qty} shares of {activity.symbol} at {activity.price} on {activity.transaction_time}")
else:
    print("No recent activities.")

Symbol: SPY, Side: sell, Qty: 0.01, Price: 601.032, Time: 2025-06-10 15:10:01.060081+00:00, P/L: None
Symbol: SPY, Side: buy, Qty: 5, Price: 601.09, Time: 2025-06-10 15:09:41.580214+00:00, P/L: None
Symbol: SPY, Side: buy, Qty: 1, Price: 601.15, Time: 2025-06-10 15:09:18.337749+00:00, P/L: None
Symbol: SPY, Side: buy, Qty: 0.001829801, Price: 601.158, Time: 2025-06-10 15:09:13.593033+00:00, P/L: None
Symbol: SPY, Side: buy, Qty: 0.5, Price: 601.19, Time: 2025-06-10 15:09:11.287751+00:00, P/L: None
Symbol: SPY, Side: buy, Qty: 1, Price: 601.19, Time: 2025-06-10 15:09:11.285849+00:00, P/L: None
Symbol: SPY, Side: buy, Qty: 2, Price: 601.19, Time: 2025-06-10 15:09:11.139781+00:00, P/L: None
Symbol: SPY, Side: buy, Qty: 2, Price: 601.21, Time: 2025-06-10 15:09:10.868010+00:00, P/L: None
Symbol: SPY, Side: sell, Qty: 0.01, Price: 600.982, Time: 2025-06-10 15:00:51.865423+00:00, P/L: None
Symbol: SPY, Side: buy, Qty: 1, Price: 601.02, Time: 2025-06-10 15:00:25.313495+00:00, P/L: None
Symbol:

In [18]:
def fetch_open_orders(api, symbol='SPY') -> list:
    """Fetches and returns open orders for a specific symbol."""
    try:
        orders = api.list_orders(status='open', symbols=[symbol])
        if orders:
            for order in orders:
                print(f"Order ID: {order.id}, Symbol: {order.symbol}, Qty: {order.qty}, Side: {order.side}, Type: {order.type}, Status: {order.status}")
        else:
            print(f"No open orders for symbol {symbol}.")
        return orders
    except Exception as e:
        print(f"Error fetching open orders: {e}")
        return []


In [19]:
open_orders = fetch_open_orders(api)

No open orders for symbol SPY.


In [20]:
def fetch_market_clock(api):
    """Fetches and prints the current market clock information."""
    try:
        clock = api.get_clock()
        print(f"Market is currently {'open' if clock.is_open else 'closed'}. Next open: {clock.next_open}, Next close: {clock.next_close}")
        return clock
    except Exception as e:
        print(f"Error fetching market clock: {e}")
        return None

In [22]:
fetch_market_clock(api)

Market is currently open. Next open: 2025-06-16 09:30:00-04:00, Next close: 2025-06-13 16:00:00-04:00


Clock({   'is_open': True,
    'next_close': '2025-06-13T16:00:00-04:00',
    'next_open': '2025-06-16T09:30:00-04:00',
    'timestamp': '2025-06-13T13:55:47.061261004-04:00'})

In [23]:
def fetch_market_calendar(api, start_date, end_date) -> list:
    """Fetches and returns the market calendar between specified dates."""
    try:
        calendar = api.get_calendar(start=start_date, end=end_date)
        if calendar:
            for day in calendar:
                print(f"Date: {day.date}, Open: {day.open}, Close: {day.close}")
        else:
            print("No market calendar data found for the specified dates.")
        return calendar
    except Exception as e:
        print(f"Error fetching market calendar: {e}")
        return []

In [24]:
fetch_market_calendar(api, start_date=datetime.now().date(), end_date=(datetime.now() + timedelta(days=7)).date())

Date: 2025-06-13 00:00:00, Open: 09:30:00, Close: 16:00:00
Date: 2025-06-16 00:00:00, Open: 09:30:00, Close: 16:00:00
Date: 2025-06-17 00:00:00, Open: 09:30:00, Close: 16:00:00
Date: 2025-06-18 00:00:00, Open: 09:30:00, Close: 16:00:00
Date: 2025-06-20 00:00:00, Open: 09:30:00, Close: 16:00:00


[Calendar({   'close': '16:00',
     'date': '2025-06-13',
     'open': '09:30',
     'session_close': '2000',
     'session_open': '0400',
     'settlement_date': '2025-06-16'}),
 Calendar({   'close': '16:00',
     'date': '2025-06-16',
     'open': '09:30',
     'session_close': '2000',
     'session_open': '0400',
     'settlement_date': '2025-06-17'}),
 Calendar({   'close': '16:00',
     'date': '2025-06-17',
     'open': '09:30',
     'session_close': '2000',
     'session_open': '0400',
     'settlement_date': '2025-06-18'}),
 Calendar({   'close': '16:00',
     'date': '2025-06-18',
     'open': '09:30',
     'session_close': '2000',
     'session_open': '0400',
     'settlement_date': '2025-06-20'}),
 Calendar({   'close': '16:00',
     'date': '2025-06-20',
     'open': '09:30',
     'session_close': '2000',
     'session_open': '0400',
     'settlement_date': '2025-06-23'})]

In [38]:
def get_historical_data_client() -> StockHistoricalDataClient:
    """Creates and returns a StockHistoricalDataClient instance."""
    api_key = os.getenv("ALPACA_API_KEY")
    api_secret = os.getenv("ALPACA_SECRET_KEY")
    if api_key and api_secret:
        return StockHistoricalDataClient(api_key, api_secret)
    else:
        raise ValueError("API key and secret must be provided.")

In [39]:
def build_stock_bars_request(symbol: str, start: datetime, end: datetime, timeframe: TimeFrame) -> StockBarsRequest:
    """Builds and returns a StockBarsRequest object."""
    return StockBarsRequest(
        symbol_or_symbols=[symbol],
        start=start,
        end=end,
        timeframe=timeframe
    )
    if feed:
        params['feed'] = feed
    return StockBarsRequest(**params)

In [51]:
def fetch_historical_data(symbol, start, end, timeframe) -> pd.DataFrame | None:
    """Fetches historical stock data from Alpaca and returns a Polars DataFrame."""
    try:
        client = get_historical_data_client()
        request_params = build_stock_bars_request(symbol, start, end, timeframe)
        bars = client.get_stock_bars(request_params)
        df = bars.df
        if df.empty:
            print("No historical data returned.")
            return None
        df.reset_index(inplace=True)  # Make sure 'timestamp' is a column
        # Set 'timestamp' as the index before converting to polars
        df = df.set_index('timestamp')
        df = df.sort_index()
        return df
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        return None


In [53]:
fetch_historical_data('SPY', datetime.now() - timedelta(days=5), datetime.now(), TimeFrame.Day)

,symbol,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2025-06-09 04:00:00+00:00,SPY,599.72,601.25,598.49,599.68,53016418.0,637329.0,600.053937
2025-06-10 04:00:00+00:00,SPY,600.22,603.47,599.09,603.08,66246973.0,674474.0,601.631918
2025-06-11 04:00:00+00:00,SPY,604.19,605.06,599.27,601.36,73658225.0,808639.0,602.345297
2025-06-12 04:00:00+00:00,SPY,600.01,603.75,599.52,603.75,64129003.0,689902.0,602.541248
2025-06-13 04:00:00+00:00,SPY,598.50,601.85,596.04,596.41,53683728.0,673936.0,599.222196


In [ ]:
def create_trade_log_file():
    """
    Creates a trade_log.csv file with headers in the same directory as the notebook.
    """
    trade_log_file = 'trade_log.csv'
    headers = "entry_time,exit_time,side,entry_price,exit_price,pnl,logged_at\n"
    if not os.path.exists(trade_log_file):
        with open(trade_log_file, 'w') as f:
            f.write(headers)
        print(f"Created {trade_log_file} with headers.")
    else:
        print(f"{trade_log_file} already exists.")

In [58]:
def log_trade(entry_time, exit_time, side, entry_price, exit_price, pnl):
    """Logs trade details to a CSV file."""
    TRADE_LOG_FILE = 'trade_log.csv'
    headers = ["entry_time", "exit_time", "side", "entry_price", "exit_price", "pnl", "logged_at"]
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    file_exists = os.path.exists(TRADE_LOG_FILE)

    with open(TRADE_LOG_FILE, 'a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(headers)
        writer.writerow([entry_time, exit_time, side, entry_price, exit_price, pnl, timestamp])
    print("Trade logged.")


In [ ]:
def place_order(symbol, side, qty, price):
    try:
        api.submit_order(
        symbol=symbol,
        qty=qty,
        side=side,
        type='limit',
        limit_price=round(price, 2),
        time_in_force='gtc')
        print(f"{side.upper()} order placed at {price}")
    except Exception as e:
        print(f"Order error: {e}")

In [60]:
# Use the existing pandas DataFrame 'df' for the latest price and order placement
df = fetch_historical_data(SYMBOL, datetime.now() - timedelta(days=1), datetime.now(), TimeFrame.Minute)
if df is not None and not df.empty:
    latest_price = float(df['close'].iloc[-1])
    print(f"Latest price for {SYMBOL}: {latest_price}")
    place_order(SYMBOL, 'sell', 1, latest_price)
else:
    print("No recent bars found.")

Latest price for SPY: 599.22
SELL order placed at 599.22
Order error: log_trade() missing 2 required positional arguments: 'exit_price' and 'pnl'


In [13]:
def check_daily_limits():
    #List comphrension to a list of daily_list
    daily_trades = [activity for activity in activities if datetime.fromisoformat(activity["transaction_time"].isoformat()).date() == datetime.now().date()]
    print(f"daily trades: {daily_trades}")
    # Code into long way: for t in daily_trades if t.side in ['buy', 'sell']]

    filtered_trades = []
    for t in daily_trades:
        if t['side'] in ['buy', 'sell']:
            filtered_trades.append(t)
    # Lets sum all pnl for today:
    today_pnl = sum(trade["realized_pl"] for trade in filtered_trades)
    print(f"Today's total PNL: {today_pnl}")

    try:
        trade_count = len([t for t in daily_trades if t['side'] in ['buy', 'sell']])
        print(f"Trade count: {trade_count}")
        today_pnl = sum(trade["realized_pl"] for trade in filtered_trades)
        print(f"Today's total PNL: {today_pnl}")
        print(f"returning: {trade_count, today_pnl}")
        return trade_count, today_pnl
    except Exception as e:
        print(f"Runtime error {e}")
        return 0, 0

In [ ]:
def get_data(symbol, timeframe='1m', limit=30):
    try:
        api_key = os.getenv('ALPACA_API_KEY')
        secret_key = os.getenv('ALPACA_SECRET_KEY')
        base_url = os.getenv('ALPACA_API_BASE_URL')

        if not api_key or not secret_key:
            print("Alpaca API keys not found. Please set ALPACA_API_KEY and ALPACA_SECRET_KEY.")
            return pd.DataFrame()

        # Compute time range
        end = datetime.utcnow()
        start = end - timedelta(minutes=limit)

        # Choose correct timeframe
        if timeframe == '1m':
            tf = TimeFrame(1, TimeFrameUnit.Minute)
        elif timeframe == '5m':
            tf = TimeFrame(5, TimeFrameUnit.Minute)
        else:
            raise ValueError(f"Unsupported timeframe: {timeframe}")

        # Build request
        request_params = StockBarsRequest(
            symbol_or_symbols=symbol,
            timeframe=tf,
            start=start,
            end=end,
            feed='iex'
        )

        # Create data client for historical stock data
        client = StockHistoricalDataClient(api_key, secret_key)

        # Fetch and process data
        bars = client.get_stock_bars(request_params).df
        df = bars.reset_index().tail(limit)
        df.set_index('timestamp', inplace=True)

        return df

    except Exception as e:
        print(f"Data error: {e}")
        return pd.DataFrame()


In [29]:
df = get_data(SYMBOL, '1m', 30)
df

,symbol,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2025-06-12 20:45:00+00:00,TSLA,318.78,318.79,318.78,318.79,234.0,5.0,318.785


In [15]:
# Interactive chart with plotly
def create_candlestick_chart(df):
    # Create candlestick trace
    candlestick = go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close']
    )

    # Create figure and add trace
    fig = go.Figure(data=[candlestick])

    # Update layout
    fig.update_layout(
        title=f'{SYMBOL} Interactive Candlestick Chart',
        xaxis_title='Time',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False # Hide range slider
    )

    # Show the plot
    fig.show()

In [16]:
def calculate_ema(data, period=9):
    return data['Close'].ewm(span=period, adjust=False).mean()

In [17]:
def calculate_rsi(data, period=14):
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

In [18]:
def average_volume(data, window=20):
    return data['Volume'].rolling(window=window).mean()

In [19]:
def calculate_slope(series, window=5):
    return series.diff().rolling(window=window).mean()

In [20]:
def check_missing_values(df):
    missing_values = df.isnull().sum()
    missing_values = missing_values[missing_values > 0]
    return missing_values

In [21]:
# Remove columns 'Dividends' and 'Stock Splits'
def remove_dividends_splits(df, columns):
    try:
        df = df.drop(columns=columns)
    except Exception as e:
        print(f"Data error: {e}")
    return df

In [22]:
def trade():

    position = load_position()
    print(f"position: {position}")

    # --- Modified Cooldown Check ---
    now = datetime.now()
    print(f"now: {now}")

    # Check if last_exit is a non-empty string before attempting to parse
    last_exit_str = position.get('last_exit', '') # Use .get to safely access, default to ''
    print(f"position['last_exit'] string: '{last_exit_str}'")

    if last_exit_str: # Check if the string is not empty
        try:
            last_exit = datetime.fromisoformat(last_exit_str)
            print(f"last_exit (parsed): {last_exit}")

            # Cooldown logic only applies if last_exit was successfully parsed
            formatted_time = last_exit.strftime("%H:%M:%S")
            print(f"formatted_time: {formatted_time}")
            time_from_last_exit = int((now - last_exit).total_seconds())
            print(f"time_from_last_exit: {time_from_last_exit}")
            time_left = COOLDOWN - time_from_last_exit
            print(f"time_left: {time_left}")

            if (time_left < COOLDOWN) and (time_left > 0):
                print(f"Cooldown active until {time_left} seconds from now. Last trade was at {formatted_time}")
                return # Exit the trade function if cooldown is active
        except ValueError:
            # Handle cases where the string is not a valid isoformat
            print(f"Warning: 'last_exit' value '{last_exit_str}' is not a valid ISO format. Assuming no recent exit for cooldown.")
            # Treat as if there was no recent exit (cooldown not active)
            time_left = COOLDOWN + 1 # Ensure time_left is greater than COOLDOWN
    else:
        # If last_exit_str is empty, assume no recent exit (cooldown not active)
        print("'last_exit' is empty. Assuming no recent exit for cooldown.")
        time_left = COOLDOWN + 1 # Ensure time_left is greater than COOLDOWN

    # Rest of your trade logic continues here if cooldown is not active
    trade_count, daily_pnl = check_daily_limits()
    if trade_count >= MAX_TRADES_PER_DAY or daily_pnl <= -DAILY_LOSS_LIMIT:
        print("Daily limit reached.")
        return

    df_1m = get_data(SYMBOL, '1m', 30)
    df_5m = get_data(SYMBOL, '5m', 30)

    if df_1m.empty or df_5m.empty:
        print("Data unavailable.")
        return

    df_1m['ema9'] = calculate_ema(df_1m)
    df_1m['rsi'] = calculate_rsi(df_1m)
    df_1m['avg_volume'] = average_volume(df_1m)

    df_5m['ema50'] = calculate_ema(df_5m, 50)
    df_5m['slope50'] = calculate_slope(df_5m['ema50'])

    last = df_1m.iloc[-1].copy()
    #print("Raw last row values:")
    for k, v in last.items():
        print(f"  {k}: {v}")

    # convert the values in the last row of that DataFrame to a consistent numeric format.
    numerical_columns = df_1m.select_dtypes(include='number').columns.tolist()
    #print(f"Numeric columns: {numerical_columns}")
    for col in numerical_columns:
        try:
            val = last[col]
            if pd.isna(val) or str(val).strip() in ['', 'None', 'nan', 'NaN']:
                raise ValueError(f"Missing or empty value for {col}")
                val = float(val) if isinstance(val, (int, float)) else float(str(val).replace(',', '').strip())
                last[col] = val
        except Exception as e:
            print(f"Data conversion error for '{col}': {e}")

    #last

    slope = float(df_5m['slope50'].iloc[-1])
    print(f"Slope: {slope}")
    side = position.get('side')
    print(f"Side: {side}")
    qty = position.get('qty', 0)
    print(f"Qty: {qty}")
    entry_price = position.get('entry_price', 0)
    print(f"Entry price: {entry_price}")
    buying_power = 100000 #float(api.get_account().cash)
    trade_qty = int((buying_power * POSITION_SIZE_PCT) / last['Close'])
    print(f"Trade qty: {trade_qty}")

    # No position
    if not side:
        print("No position found")
        save_position({'side': 'long', 'entry_price': 100, 'qty': trade_qty, 'high': 200, 'entry_time': now.isoformat()})
        # if last['Close'] > last['Open'] and last['Close'] > last['ema9'] and last['rsi'] < 65 and last['Volume'] > last['avg_volume'] and slope > 0:
        #     price = last['Close'] * 1.001
        #     place_order('buy', trade_qty, price)
        #     save_position({'side': 'long', 'entry_price': price, 'qty': trade_qty, 'high': price, 'entry_time': now.isoformat()})

        # elif last['Close'] < last['Open'] and last['Close'] < last['ema9'] and last['rsi'] > 55 and last['Volume'] > last['avg_volume'] and slope < 0:
        #     price = last['Close'] * 0.999
        #     place_order('sell', trade_qty, price)
        #     save_position({'side': 'short', 'entry_price': price, 'qty': trade_qty, 'low': price, 'entry_time': now.isoformat()})

    # Long Position
    elif side == 'long':
        print("Long Position")
        position['high'] = max(position['high'], last['Close'])
        trail_stop = position['high'] * (1 - TRAILING_STOP_PCT)
        print(f"[LONG] Comparing last['Close'] = {last['Close']} (type: {type(last['Close'])}) to trail_stop = {trail_stop}")
        last_close = float(last['Close'])
        if last_close <= trail_stop or last_close < last['ema9']:
            close_position()
            exit_price = last['Close']
            pnl = (exit_price - entry_price) * qty
            log_trade(position['entry_time'], now.isoformat(), side, entry_price, exit_price, pnl)
            #reset_position()
            save_position({'last_exit': now.isoformat()})

    # Short Position
    elif side == 'short':
        print("Short Position")
        position['low'] = min(position['low'], last['Close'])
        trail_stop = position['low'] * (1 + TRAILING_STOP_PCT)
        print(f"[SHORT] Comparing last['Close'] = {last['Close']} (type: {type(last['Close'])}) to trail_stop = {trail_stop}")
        last_close = float(last['Close'])
        if last_close >= trail_stop or last_close > last['ema9']:
            close_position()
            exit_price = last['Close']
            pnl = (entry_price - exit_price) * qty
            log_trade(position['entry_time'], now.isoformat(), side, entry_price, exit_price, pnl)
            #reset_position()
            save_position({'last_exit': now.isoformat()})


In [52]:
try:
    trade()
except Exception as e:
    print(f"Runtime error: {e}")
#time.sleep(60)

'position.json' already exists.
Successfully loaded position from position.json
position: {'last_exit': '2025-06-12T20:34:21.735676'}
now: 2025-06-12 20:34:33.084023
position['last_exit'] string: '2025-06-12T20:34:21.735676'
last_exit (parsed): 2025-06-12 20:34:21.735676
formatted_time: 20:34:21
time_from_last_exit: 11
time_left: 9
Cooldown active until 9 seconds from now. Last trade was at 20:34:21
